<a href="https://colab.research.google.com/github/gracelcai/career-launch-group-13/blob/main/exercise_classification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Import PyTorch and other libraries

In [ ]:
import torch
import numpy as np

# 1. Download and process Kaggle data

In [ ]:
!pip install -q opendatasets

100%|██████████| 1.07M/1.07M [00:00<00:00, 93.9MB/s]

In [ ]:
import opendatasets as od
import pandas as pd

od.download('https://www.kaggle.com/datasets/muhannadtuameh/exercise-recognition/data') # insert ypu kaggle  username and key

distances_data = pd.read_csv('/content/exercise-recognition/3d_distances.csv')
angles_data = pd.read_csv('/content/exercise-recognition/angles.csv')
landmarks_data = pd.read_csv('/content/exercise-recognition/landmarks.csv')
xyz_distances_data = pd.read_csv('/content/exercise-recognition/xyz_distances.csv')

labels_data = pd.read_csv('/content/exercise-recognition/labels.csv')['pose'].tolist()

Skipping, found downloaded files in "./exercise-recognition" (use force=True to force download)


In [ ]:
distances_data.head()

,pose_id,left_shoulder_left_wrist,right_shoulder_right_wrist,left_hip_left_ankle,right_hip_right_ankle,left_hip_left_wrist,right_hip_right_wrist,left_shoulder_left_ankle,right_shoulder_right_ankle,left_hip_right_wrist,right_hip_left_wrist,left_elbow_right_elbow,left_knee_right_knee,left_wrist_right_wrist,left_ankle_right_ankle,left_hip_avg_left_wrist_left_ankle,right_hip_avg_right_wrist_right_ankle
0,0,38.423300,39.855762,62.640720,57.871964,15.052525,20.688140,107.80279,101.256220,25.964552,20.129477,21.891373,9.572068,23.628513,8.817467,24.935423,21.897507
1,1,65.631510,65.847110,55.631300,53.715553,106.099660,99.581940,108.05316,108.368080,99.120370,111.111630,34.513107,13.026263,39.510204,10.478581,70.999214,66.304980
2,2,69.661674,69.360990,55.770256,53.510430,110.819440,109.959590,103.96043,102.363594,109.834720,115.859040,37.486490,14.406796,43.056570,9.337358,75.719420,73.916530
3,3,52.437600,55.870330,57.758915,57.445232,97.575290,94.503000,119.82603,118.113594,92.485230,105.289980,38.071854,12.797712,50.756527,9.881827,60.924847,59.798840
4,4,30.867628,43.872227,67.210240,65.753365,25.952522,44.106445,105.16242,102.763930,44.958740,31.483402,22.992490,6.848766,36.983727,5.223047,23.287280,19.660568


In [ ]:
# turn string labels into class numbers
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
class_names = ["pushups_up", "pushups_down", "pullups_up", "pullups_down", "jumping_jacks_up", "jumping_jacks_down", "situp_up", "situp_down", "squats_up", "squats_down"]

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels_data)
labels

array([0, 0, 0, ..., 9, 9, 9])

In [ ]:
# turn data into tensors - distances only
X = torch.from_numpy(distances_data.to_numpy()[:, 1:]).type(torch.float)
y = torch.from_numpy(labels).type(torch.float)
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

X shape: torch.Size([1372, 16])
y shape: torch.Size([1372])


In [ ]:
X[:5], y[:5]

(tensor([[ 38.4233,  39.8558,  62.6407,  57.8720,  15.0525,  20.6881, 107.8028,
          101.2562,  25.9646,  20.1295,  21.8914,   9.5721,  23.6285,   8.8175,
           24.9354,  21.8975],
         [ 65.6315,  65.8471,  55.6313,  53.7156, 106.0997,  99.5819, 108.0532,
          108.3681,  99.1204, 111.1116,  34.5131,  13.0263,  39.5102,  10.4786,
           70.9992,  66.3050],
         [ 69.6617,  69.3610,  55.7703,  53.5104, 110.8194, 109.9596, 103.9604,
          102.3636, 109.8347, 115.8590,  37.4865,  14.4068,  43.0566,   9.3374,
           75.7194,  73.9165],
         [ 52.4376,  55.8703,  57.7589,  57.4452,  97.5753,  94.5030, 119.8260,
          118.1136,  92.4852, 105.2900,  38.0719,  12.7977,  50.7565,   9.8818,
           60.9248,  59.7988],
         [ 30.8676,  43.8722,  67.2102,  65.7534,  25.9525,  44.1064, 105.1624,
          102.7639,  44.9587,  31.4834,  22.9925,   6.8488,  36.9837,   5.2230,
           23.2873,  19.6606]]),
 tensor([0., 0., 0., 0., 0.]))

In [ ]:
type(X), X.dtype, y.dtype

(torch.Tensor, torch.float32, torch.float32)

In [ ]:
# split data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2, # percentage of test
                                                    random_state=42)
X_train.shape, y_train.shape

(torch.Size([1097, 16]), torch.Size([1097]))

In [ ]:
len(X_train), len(X_test)

(1097, 275)

# 2. Model

In [ ]:
import torch
from torch import nn

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [ ]:
class DistanceModelV0(nn.Module):
  def __init__(self, input_features, output_features, hidden_units=8):
    super().__init__()

    self.layer_stack = nn.Sequential(
        nn.Linear(in_features=input_features, out_features=hidden_units),
        nn.ReLU(),
        nn.Linear(in_features=hidden_units, out_features=hidden_units),
        nn.ReLU(),
        nn.Linear(in_features=hidden_units, out_features=output_features)
    )

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    return self.layer_stack(x)

distance_model_0 = DistanceModelV0(input_features=16,
                                   hidden_units=200,
                                   output_features=10).to(device)
distance_model_0

DistanceModelV0(
  (layer_stack): Sequential(
    (0): Linear(in_features=16, out_features=200, bias=True)
    (1): ReLU()
    (2): Linear(in_features=200, out_features=200, bias=True)
    (3): ReLU()
    (4): Linear(in_features=200, out_features=10, bias=True)
  )
)

# 3. Train distance model

## 3.1 Functions

In [ ]:
# calculate accuracy
def accuracy_fn(y_true, y_pred):
  correct = torch.eq(y_true, y_pred).sum().item()
  acc = (correct/len(y_pred)) * 100
  return acc

In [ ]:
# loss function and optimizer
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(params=distance_model_0.parameters(),
                            lr=0.01)

In [ ]:
# fit the multi-class model to the data
torch.manual_seed(42)
torch.cuda.manual_seed(42)

epochs = 200

X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

for epoch in range(epochs):
  distance_model_0.train()

  y_logits = distance_model_0(X_train)
  y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1)

  loss = loss_fn(y_logits, y_train.type(torch.LongTensor))
  acc = accuracy_fn(y_true=y_train,
                    y_pred=y_pred)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  ### Testing
  distance_model_0.eval()
  with torch.inference_mode():
    test_logits = distance_model_0(X_test)
    test_pred = torch.softmax(test_logits, dim=1).argmax(dim=1)

    test_loss = loss_fn(test_logits, y_test.type(torch.LongTensor))
    test_acc = accuracy_fn(y_true=y_test,
                      y_pred=test_pred)

    if epoch % 10 == 0:
      print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Test loss: {test_loss:.5f}, Test accuracy: {test_acc:.2f}%")

Epoch: 0 | Loss: 0.87565, Accuracy: 69.74% | Test loss: 1.27044, Test accuracy: 58.18%
Epoch: 10 | Loss: 0.81950, Accuracy: 73.75% | Test loss: 1.22899, Test accuracy: 61.45%
Epoch: 20 | Loss: 0.85160, Accuracy: 72.56% | Test loss: 1.29818, Test accuracy: 58.91%
Epoch: 30 | Loss: 0.82921, Accuracy: 73.38% | Test loss: 1.24355, Test accuracy: 61.82%
Epoch: 40 | Loss: 0.79744, Accuracy: 74.20% | Test loss: 1.22299, Test accuracy: 62.91%
Epoch: 50 | Loss: 0.77633, Accuracy: 74.29% | Test loss: 1.21425, Test accuracy: 63.27%
Epoch: 60 | Loss: 0.75679, Accuracy: 74.84% | Test loss: 1.20750, Test accuracy: 63.64%
Epoch: 70 | Loss: 0.79909, Accuracy: 71.47% | Test loss: 1.24624, Test accuracy: 57.09%
Epoch: 80 | Loss: 0.73660, Accuracy: 76.48% | Test loss: 1.21740, Test accuracy: 62.55%
Epoch: 90 | Loss: 0.72584, Accuracy: 77.03% | Test loss: 1.21688, Test accuracy: 64.00%
Epoch: 100 | Loss: 0.73602, Accuracy: 76.85% | Test loss: 1.29781, Test accuracy: 59.64%
Epoch: 110 | Loss: 0.72188, Accu